In [61]:
q = 23
F = GF(q)
F_odd = [F(i) for i in range(q) if i % 2 != 0]
F_even = [F(i) for i in range(q) if i % 2 == 0]
 
def gen_root_mod(seed):
    v = hashlib.sha3_256(seed).digest()
    t = F(hashlib.sha3_256(seed + 'flag'.encode()).digest()[0]) 
    
    return v, t

In [123]:
depth = 5
N = 2**depth
coord = randint(0, N-1)
target = F.random_element()
print('coord: ', coord)
print('target: ', target)

coord:  27
target:  16


In [124]:
def path(num, d=depth):
    """
    Convert an integer to a binary array using bit manipulation.
    
    Args:
        num (int): The input integer (must be ≥ 0).
        d (int): The desired length of the binary array.
    
    Returns:
        list: Binary array (e.g., 5 with d=4 → `[0, 1, 0, 1]`).
    """
    if num < 0:
        raise ValueError("Input must be non-negative")
    if d < 0:
        raise ValueError("Length must be positive")
    if num >= (1 << d):  # Check if number is too large for d bits
        raise ValueError(f"{num} requires more than {d} bits")
    
    return [(num >> (d - 1 - i)) & 1 for i in range(d)]

p = path(coord, d=depth)
p

[1, 1, 0, 1, 1]

In [125]:
import hashlib

def gen_descendants_mod(v, t, cw):
    v_left = hashlib.sha3_256(v + 'l1'.encode()).digest()
    t_left = F(hashlib.sha3_256(v + 'l2'.encode()).digest()[0])

    v_right = hashlib.sha3_256(v + 'r1'.encode()).digest()
    t_right = F(hashlib.sha3_256(v + 'r2'.encode()).digest()[0])

    if Integer(t) % 2 != 0:
    #if False:
        v_left = bytes([v_left[i] ^^ cw[0][i] for i in range(32)])
        t_left = F(t_left + cw[1])
        
        v_right = bytes([v_right[i] ^^ cw[2][i] for i in range(32)])
        t_right = F(t_right + cw[3])

    #print('gen_descendants: ', t_left, t_right)
    return v_left, t_left, v_right, t_right

In [131]:
def find_cw_at_ith_level_mod(i, v_a_i, t_a_i, v_b_i, t_b_i, p, last=False, val=None):
    v_a1_left, t_a1_left, v_a1_right, t_a1_right = gen_descendants_mod(v_a_i, F(0), [])
    v_b1_left, t_b1_left, v_b1_right, t_b1_right = gen_descendants_mod(v_b_i, F(0), [])
    
    if p[i] == 0:
        fl1 = Integer(t_a1_left) % 2 == 0
        fl2 = Integer(t_b1_left) % 2 == 0
        
        # will be added to left descendants
        cw0 = bytes([0]*32)
        if last:
            if Integer(t_a_i) % 2 != 0:
                cw1 = cw1 = F(val + t_b1_left - t_a1_left) 
                
                cw3 = F(-t_a1_right + t_b1_right) 
            elif Integer(t_b_i) % 2 != 0:
                cw1 = F(val - t_b1_left + t_a1_left)
                
                cw3 = F(-t_b1_right + t_a1_right) 
                
        elif Integer(t_a_i) % 2 != 0:
            w = choice(F_odd) if fl2 else choice(F_even)
            cw1 = w - t_a1_left

            cw3 = F(-t_a1_right + t_b1_right)
        elif Integer(t_b_i) % 2 != 0:
            w = choice(F_odd) if fl1 else choice(F_even)
            cw1 = w - t_b1_left  

            cw3 = F(-t_b1_right + t_a1_right)
            
        # will be added to right descendants
        cw2 = bytes([v_a1_right[i] ^^ v_b1_right[i] for i in range(32)])
        #cw3 = F(-t_b1_right - t_a1_right)
    
    if p[i] == 1:
        fl1 = Integer(t_a1_right) % 2 == 0
        fl2 = Integer(t_b1_right) % 2 == 0
        
        # will be added to left descendants
        cw0 = bytes([v_a1_left[i] ^^ v_b1_left[i] for i in range(32)])
        #cw1 = F(t_b1_left + t_a1_left)
    
        # will be added to right descendants
        cw2 = bytes([0]*32) 

        if last:
            if Integer(t_a_i) % 2 != 0:
                cw1 = F(-t_a1_left + t_b1_left) 
                
                cw3 = F(val + t_b1_right - t_a1_right)    
            elif Integer(t_b_i) % 2 != 0:
                cw1 = F(-t_b1_left + t_a1_left) 
                
                cw3 = F(val - t_b1_right + t_a1_right)    
                
        elif Integer(t_a_i) % 2 != 0:
            cw1 = F(-t_a1_left + t_b1_left)
            
            w = choice(F_odd) if fl2 else choice(F_even)
            cw3 = w - t_a1_right
        elif Integer(t_b_i) % 2 != 0:
            cw1 = F(-t_b1_left + t_a1_left)
            
            w = choice(F_odd) if fl1 else choice(F_even)
            cw3 = w - t_b1_right  
    
    return [cw0, cw1, cw2, cw3]
    #return [cw0, F(1), cw2, F(1)]

In [132]:
import secrets

seed_a = secrets.token_bytes(32)
v_a, t_a = gen_root_mod(seed_a)

seed_b = secrets.token_bytes(32)
v_b, t_b = gen_root_mod(seed_b)

while Integer(t_a) % 2 == Integer(t_b) % 2:
    seed_b = secrets.token_bytes(32)
    v_b, t_b = gen_root_mod(seed_b) 

print(t_a, t_b)

CW_mod = []

for i in range(depth):
    last = (i == depth - 1)
    cw = find_cw_at_ith_level_mod(i, v_a, t_a, v_b, t_b, p, last=last, val=target)
    CW_mod.append(cw)

    v_a_left_, t_a_left_, v_a_right_, t_a_right_ = gen_descendants_mod(v_a, t_a, cw)
    v_b_left_, t_b_left_, v_b_right_, t_b_right_ = gen_descendants_mod(v_b, t_b, cw)
    
    if p[i] == 0:
        v_a, t_a, v_b, t_b = v_a_left_, t_a_left_, v_b_left_, t_b_left_
    else:
        v_a, t_a, v_b, t_b = v_a_right_, t_a_right_, v_b_right_, t_b_right_

0 19


In [133]:
def compute_at_i_mod(seed, CW, i, depth=depth):
    v, t = gen_root_mod(seed)
    p = path(i, d=depth)
    
    for j in range(depth):
        v_left_, t_left_, v_right_, t_right_ = gen_descendants_mod(v, t, CW[j])
        #print('compute_at_i: ', t_left_, t_right_)
        if p[j] == 0:
            v, t = v_left_, t_left_
        else:
            v, t = v_right_, t_right_

    return t

In [135]:
print(p)

for i in CW_mod:
    print(i[1], i[3])

for i in range(depth+1):
    l = []
    for j in range(2**i):
        l.append(compute_at_i_mod(seed_a, CW_mod, j, i))
    print(l)

print('##########################')

for i in range(depth+1):
    l = []
    for j in range(2**i):
        l.append(compute_at_i_mod(seed_b, CW_mod, j, i))
    print(l)

print('##########################')
for i in range(depth+1):
    l = []
    for j in range(2**i):
        l.append(compute_at_i_mod(seed_a, CW_mod, j, i) - compute_at_i_mod(seed_b, CW_mod, j, i))
    print(l)

[1, 1, 0, 1, 1]
3 9
22 12
12 1
21 4
13 20
[0]
[17, 3]
[11, 15, 18, 5]
[6, 0, 0, 3, 5, 1, 17, 18]
[2, 0, 12, 3, 10, 5, 10, 10, 5, 14, 4, 20, 6, 13, 1, 1]
[0, 18, 5, 22, 4, 11, 15, 14, 15, 8, 21, 5, 13, 4, 2, 2, 7, 21, 4, 13, 19, 10, 7, 2, 9, 19, 13, 17, 7, 4, 11, 6]
##########################
[19]
[17, 4]
[11, 15, 18, 12]
[6, 0, 0, 3, 5, 1, 12, 18]
[2, 0, 12, 3, 10, 5, 10, 10, 5, 14, 4, 20, 6, 14, 1, 1]
[0, 18, 5, 22, 4, 11, 15, 14, 15, 8, 21, 5, 13, 4, 2, 2, 7, 21, 4, 13, 19, 10, 7, 2, 9, 19, 13, 1, 7, 4, 11, 6]
##########################
[4]
[0, 22]
[0, 0, 0, 16]
[0, 0, 0, 0, 0, 0, 5, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 22, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0]


In [137]:
print(f'i\ta\tb\tF(a + b)')
for i in range(N):
    a = compute_at_i_mod(seed_a, CW_mod, i)
    b = compute_at_i_mod(seed_b, CW_mod, i)
    print(f'{i}\t{a}\t{b}\t{F(a - b)}')

i	a	b	F(a + b)
0	0	0	0
1	18	18	0
2	5	5	0
3	22	22	0
4	4	4	0
5	11	11	0
6	15	15	0
7	14	14	0
8	15	15	0
9	8	8	0
10	21	21	0
11	5	5	0
12	13	13	0
13	4	4	0
14	2	2	0
15	2	2	0
16	7	7	0
17	21	21	0
18	4	4	0
19	13	13	0
20	19	19	0
21	10	10	0
22	7	7	0
23	2	2	0
24	9	9	0
25	19	19	0
26	13	13	0
27	17	1	16
28	7	7	0
29	4	4	0
30	11	11	0
31	6	6	0
